# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
# Group the dataframe by 'CustomerID' and 'ProductID' and sum the 'Quantity' field
group = data.groupby(by=['CustomerID', 'ProductName']).sum()['Quantity'].reset_index()

# Check the result
group

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
# Create a pivot table
pt = pd.pivot_table(group, values='Quantity', index='ProductName', columns='CustomerID', fill_value=0)

# Check the result
pt

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
# Create a customer similarity matrix
sim_matrix = pd.DataFrame((1 / (1 + squareform(pdist(pt.T, metric='euclidean')))), index=pt.columns, columns=pt.columns)

# Check the result
sim_matrix

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
# Necessary variable
customer_id = 33

# List with top 5 most similar customers for a specific ID
top5_similar = list(sim_matrix[customer_id].sort_values(ascending=False)[1:6].index)

# Check the result using as examples 'CustomerID' = 33
top5_similar

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
# Filter the dataset 'group'
list_prod_recomm_all = group[group.CustomerID.isin(top5_similar)]

# Check the result
list_prod_recomm_all

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1
...,...,...,...
3003,3535,Tray - 16in Rnd Blk,1
3004,3535,Wanton Wrap,1
3005,3535,Wine - Blue Nun Qualitatswein,1
3006,3535,"Wine - Magnotta, Merlot Sr Vqa",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
# Aggregate products and rank them by 'Quantity'
list_recomm_rank = list_prod_recomm_all.groupby(by='ProductName').sum()['Quantity'].sort_values(ascending=False).reset_index()

# Check the result
list_recomm_rank

,ProductName,Quantity
0,Butter - Unsalted,3
1,Soup - Campbells Bean Medley,3
2,Towels - Paper / Kraft,3
3,Wine - Ej Gallo Sierra Valley,3
4,"Pepper - Black, Whole",3
...,...,...
213,"Pepper - White, Ground",1
214,"Pepper - Paprika, Hungarian",1
215,Pate - Cognac,1
216,Pastry - Choclate Baked,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
# Merge the ranked products dataframe with the customer product matrix
merged = pd.merge(list_recomm_rank, pt[customer_id], on='ProductName').rename(columns={customer_id: 'Purchased'})

# Filter the dataset
recommendation_prod = merged[merged.Purchased == 0].head()

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [11]:
# Create empty dictionary
recomendations = {}

# List of unique 'CustomerIDs'
customers_id = list(group.CustomerID.unique())

# Iterative process to add the recomendation products to the dictionary
for id in customers_id:
    # Get top 5 similar customers
    top5_similar = list(sim_matrix[id].sort_values(ascending=False)[1:6].index)
    
    # List with recommenation productions
    list_prod_recomm_all = group[group.CustomerID.isin(top5_similar)]
    list_recomm_rank = list_prod_recomm_all.groupby(by='ProductName').sum()['Quantity'].sort_values(ascending=False).reset_index()
    
    # Top 5 recommendation products
    merged = pd.merge(list_recomm_rank, pt[id], on='ProductName').rename(columns={id: 'Purchased'})
    recommendation_prod = merged[merged.Purchased == 0].head()
    
    # Add to the dictionary
    recomendations[id] = list(recommendation_prod['ProductName'])

In [12]:
# CHeck the result
recomendations

{33: ['Butter - Unsalted',
  'Soup - Campbells Bean Medley',
  'Wine - Ej Gallo Sierra Valley',
  'Wine - Blue Nun Qualitatswein',
  'Scallops 60/80 Iqf'],
 200: ['Soup - Campbells Bean Medley',
  'Muffin - Carrot Individual Wrap',
  'Bay Leaf',
  'Pork - Kidney',
  'Wanton Wrap'],
 264: ['Fish - Scallops, Cold Smoked',
  'Soupfoamcont12oz 112con',
  'Wine - Two Oceans Cabernet',
  'Bread - Italian Roll With Herbs',
  'Fondant - Icing'],
 356: ['Veal - Inside, Choice',
  'Sardines',
  'Lamb - Ground',
  'Butter - Unsalted',
  'Beef - Top Sirloin'],
 412: ['Sauce - Gravy, Au Jus, Mix',
  'Cheese - Cambozola',
  'Olive - Spread Tapenade',
  'Pepper - Black, Whole',
  'Wine - Blue Nun Qualitatswein'],
 464: ['Butter - Unsalted',
  'Yoghurt Tubes',
  'Sausage - Breakfast',
  'Sauce - Gravy, Au Jus, Mix',
  'Veal - Inside, Choice'],
 477: ['Cheese - Cambozola',
  'Veal - Inside, Choice',
  'Olive - Spread Tapenade',
  'Pepper - Black, Whole',
  'Wine - Blue Nun Qualitatswein'],
 639: ['Wine

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [13]:
# Create a new Pandas DataFrame
df_recommendation = pd.DataFrame.from_dict(recomendations, orient='index', columns=['rec1', 'rec2', 'rec3', 'rec4', 'rec5']).reset_index().rename(columns={'index': 'customer_id'})

# Check the result
df_recommendation

,customer_id,rec1,rec2,rec3,rec4,rec5
0,33,Butter - Unsalted,Soup - Campbells Bean Medley,Wine - Ej Gallo Sierra Valley,Wine - Blue Nun Qualitatswein,Scallops 60/80 Iqf
1,200,Soup - Campbells Bean Medley,Muffin - Carrot Individual Wrap,Bay Leaf,Pork - Kidney,Wanton Wrap
2,264,"Fish - Scallops, Cold Smoked",Soupfoamcont12oz 112con,Wine - Two Oceans Cabernet,Bread - Italian Roll With Herbs,Fondant - Icing
3,356,"Veal - Inside, Choice",Sardines,Lamb - Ground,Butter - Unsalted,Beef - Top Sirloin
4,412,"Sauce - Gravy, Au Jus, Mix",Cheese - Cambozola,Olive - Spread Tapenade,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein
...,...,...,...,...,...,...
995,97928,"Soup - Campbells, Lentil","Cheese - Brie,danish",Wiberg Super Cure,Pants Custom Dry Clean,Shrimp - 31/40
996,98069,Skirt - 29 Foot,Milk - 1%,Cheese - Taleggio D.o.p.,Beans - Kidney White,Sprouts - Baby Pea Tendrils
997,98159,"Lamb - Whole, Fresh",Chips Potato Salt Vinegar 43g,Tea - English Breakfast,Bread - Raisin Walnut Oval,Wine - Redchard Merritt
998,98185,Pernod,Crackers - Trio,Tea - Jasmin Green,Pastry - Choclate Baked,"Peas - Pigeon, Dry"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [14]:
# Create a customer similarity matrix
sim_matrix = pd.DataFrame((1 / (1 + squareform(pdist(pt.T, metric='cityblock')))), index=pt.columns, columns=pt.columns)

# Create empty dictionary
recomendations = {}

# List of unique 'CustomerIDs'
customers_id = list(group.CustomerID.unique())

# Iterative process to add the recomendation products to the dictionary
for id in customers_id:
    # Get top 5 similar customers
    top5_similar = list(sim_matrix[id].sort_values(ascending=False)[1:6].index)
    
    # List with recommenation productions
    list_prod_recomm_all = group[group.CustomerID.isin(top5_similar)]
    list_recomm_rank = list_prod_recomm_all.groupby(by='ProductName').sum()['Quantity'].sort_values(ascending=False).reset_index()
    
    # Top 5 recommendation products
    merged = pd.merge(list_recomm_rank, pt[id], on='ProductName').rename(columns={id: 'Purchased'})
    recommendation_prod = merged[merged.Purchased == 0].head()
    
    # Add to the dictionary
    recomendations[id] = list(recommendation_prod['ProductName'])

# Create a new Pandas DataFrame
df_recommendation = pd.DataFrame.from_dict(recomendations, orient='index', columns=['rec1', 'rec2', 'rec3', 'rec4', 'rec5']).reset_index().rename(columns={'index': 'customer_id'})

# Check the result
df_recommendation

,customer_id,rec1,rec2,rec3,rec4,rec5
0,33,Wine - Charddonnay Errazuriz,Wine - Ej Gallo Sierra Valley,"Oranges - Navel, 72",Bay Leaf,Pecan Raisin - Tarts
1,200,Scampi Tail,Pomello,Pork - Kidney,Muffin - Carrot Individual Wrap,Soup - Campbells Bean Medley
2,264,Pork - Kidney,"Veal - Inside, Choice",Black Currants,Bread - Italian Roll With Herbs,Wine - Two Oceans Cabernet
3,356,Butter - Unsalted,"Veal - Inside, Choice","Nut - Chestnuts, Whole","Beets - Candy Cane, Organic",Olives - Stuffed
4,412,Wine - Blue Nun Qualitatswein,Table Cloth 81x81 White,Butter - Unsalted,Soup - Campbells Bean Medley,"Pepper - Black, Whole"
...,...,...,...,...,...,...
995,97928,Wine - Chardonnay South,Butter - Unsalted,Raspberries - Fresh,Sausage - Liver,Coffee Decaf Colombian
996,98069,Wine - Chardonnay South,Bar - Granola Trail Mix Fruit Nut,Pork - Hock And Feet Attached,"Mushrooms - Black, Dried","Cheese - Brie, Triple Creme"
997,98159,Bandage - Fexible 1x3,Butter - Unsalted,Soup - Campbells Bean Medley,Knife Plastic - White,Sherry - Dry
998,98185,Muffin - Carrot Individual Wrap,Beef - Montreal Smoked Brisket,Knife Plastic - White,Ice Cream Bar - Hageen Daz To,Pork - Kidney
